In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import os
import json
import torch

from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset

from pnr_utils import *
from pnr_models import *
from plot_sld import plot_SLD

seed = 12
torch.set_default_dtype(torch.float64)
torch.manual_seed(seed)
np.random.seed(seed)

device = torch.device("cuda:7" if torch.cuda.is_available() else "cpu")

### Load and process synthetic data

- Input the **sample name** and the **set number** of the dataset generated by `pnr_generate.py` to locate the appropriate directories and read the metadata.


- Load and process (apply noise, smoothing, and normalization to) spectra and identify the parameter names (y_header) and corresponding indices (y_ids) to use for supervised learning.


- Standardize y-data (parameter values), perform train/valid/test split stratified by class, and convert data to torch tensors.

In [ ]:
# set data properties
sample_name = 'BiSe10_EuS5' #'CrO20_BiSbTe20' 
dataset = 0
fit_roughness = False

# get directories
set_dir = 'results/' + sample_name + '/set_' + str(dataset)
data_dir = set_dir + '/data'

# parse metadata and experiment files
layers, rho, M, N, q_min, q_max = parse_metadata(set_dir)
q = 10*np.linspace(q_min, q_max, N)
exp_names = next(os.walk('experiments/' + sample_name))[1]
exp_names = [k for k in exp_names if '-' not in k]
exp_names = [sample_name + '/' + exp_names[j] for j in np.argsort([int(i[:i.index('K')]) for i in exp_names])]

print('set directory:', set_dir)

In [ ]:
# load and process data
x_data, x_orig, x_moms, y_data, y_columns, y_header, \
y_ids, y_labels, y_units, y_units_ = process_data(data_dir, sample_name, exp_names[0], q, seed, fit_roughness)

# split data
x_train, x_valid, x_test, y_train, y_valid, y_test = split_data(x_data, y_data, seed)

In [ ]:
# standardize data
scaler = StandardScaler().fit(y_train[:,y_ids[:-1]])
torch_scaler = TorchStandardScaler()
torch_scaler.mean = torch.tensor(scaler.mean_).to(device)
torch_scaler.std = torch.tensor(np.sqrt(scaler.var_)).to(device)

y_train[:,np.array(y_ids[:-1])] = scaler.transform(y_train[:,y_ids[:-1]])
y_valid[:,np.array(y_ids[:-1])] = scaler.transform(y_valid[:,y_ids[:-1]])
y_test[:,np.array(y_ids[:-1])] = scaler.transform(y_test[:,y_ids[:-1]])

In [ ]:
# convert to torch tensors
x_train = torch.from_numpy(x_train).unsqueeze(1)
x_valid = torch.from_numpy(x_valid).unsqueeze(1)
x_test = torch.from_numpy(x_test).unsqueeze(1)
y_train = torch.from_numpy(y_train)
y_valid = torch.from_numpy(y_valid)
y_test = torch.from_numpy(y_test)

height = x_train.size()[2]
width = x_train.size()[3]
num_features = len(y_ids) - 1
prox_features = [k for k, v in enumerate(y_header) if 'prox' in v][:-1]
print('height:', height, 'width:', width, 'num_features:', num_features)

### Define model

- Input the **model name** (vae, cvae, rvae, rcvae) for the model to train according to:
    * vae: simple VAE with no classifier or regressors
    * cvae: VAE + classifier
    * rvae: VAE + regressors for each parameter value
    * rcvae: CVAE + regressors for each parameter value


- Input the **model number** to resume training (or to continue analysis without further fitting), or -1 to start a new fit.


- Input the number of models (**reps**) to train in succession, and set the **batch size** and number of **epochs**.


- Set **z_std_norm** to False if z_mean is presumed to match the parameter values, or True if presumed to be 0, and set the parameters defining the model architecture.

In [ ]:
# model to train
model_name = 'rvae'
model_num = -1            # -1 to start a new fit, model number to resume or analyze

# training parameters
reps = 1
batch_size = 256
epochs = 100

# model parameters
z_std_norm = False       # True if z ~ N(0,1), False if z ~ N(mu,1), mu > 0
kwargs = {
    'hidden_dim': 512,
    'latent_dim': 24,
    'start_filters': 16,
    'kernel_size': 7,
    'pool_size': 4,
    'num_conv': 2,
    'num_dense': 5,
    'slope': 0.3,
    'drop': False,
    'beta_1': 0.01,
    'beta_2': 0.01
}

In [ ]:
# print model architecture
model, _, _ = init_model(model_name, height, width, num_features, kwargs, device,
                         prox_features=prox_features, scaler=torch_scaler)
print(model)

print('number of parameters:', sum([p.numel() for p in model.parameters() if p.requires_grad]))

In [ ]:
# set model directories
model_prefix = '/' + model_name + '_'
resume_fit = model_num >= 0

if resume_fit:
    # load args from saved model directory
    model_dir = set_dir + model_prefix + str(model_num)
    args = torch.load(model_dir + '/model_0/model.torch', map_location='cpu')['args']
    start_epoch = args['epochs']
    epochs = start_epoch + epochs
    args['epochs'] = epochs
    
    kwargs = args['kwargs']
    
else:
    # create new model directory
    args = {}
    dirs = next(os.walk(set_dir))[1]
    for k in dirs:
        if k.startswith('.'): dirs.remove(k)
    dirs.remove('data')
    if 'properties' in dirs: dirs.remove('properties')

    if len(dirs):
        idns = [int(d.split('_')[-1]) for d in dirs]
        idn = max(idns) + 1
        args['model_num'] = idn
        model_dir = set_dir + model_prefix + str(idn)
    else:
        args['model_num'] = 0
        model_dir = set_dir + model_prefix + '0'

    os.makedirs(model_dir)
    start_epoch = 0
    
    # save arguments
    args['batch_size'] = batch_size
    args['reps'] = reps
    args['epochs'] = epochs
    args['z_std_norm'] = z_std_norm
    args['kwargs'] = kwargs.copy()
    
meta = json.dumps(args, sort_keys=True, indent=2)
with open(model_dir + '/args.txt', 'w') as f:
    f.write(meta)

print('model directory:', model_dir)

In [ ]:
# configure data loaders
data_train = TensorDataset(x_train, y_train)
data_valid = TensorDataset(x_valid, y_valid)
data_test = TensorDataset(x_test, y_test)

train_loader = DataLoader(data_train, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(data_valid, batch_size=batch_size)
test_loader = DataLoader(data_test, batch_size=batch_size)

d_sets = ['train', 'valid', 'test']
data_loaders = dict(zip(d_sets, [DataLoader(data_train, batch_size=batch_size), valid_loader, test_loader]))

### Train model

- Train reps number of models in succession.

In [ ]:
for r in range(reps):
    image_dir = model_dir + '/model_' + str(r)
    if not os.path.exists(image_dir): os.makedirs(image_dir)

    model, opt, metric_keys = init_model(model_name, height, width, num_features, kwargs, device,
                                         prox_features=prox_features, scaler=torch_scaler)

    print('====== repetition:', r)

    if resume_fit:
        model.load_state_dict(torch.load(image_dir + '/model.torch', map_location=device)['state'])
        opt.load_state_dict(torch.load(image_dir + '/model.torch', map_location=device)['optimizer'])
        dynamics = torch.load(image_dir + '/model.torch', map_location='cpu')['dynamics']
    else: dynamics = []

    for epoch in range(start_epoch + 1, epochs + 1):
        train_metrics = train(epoch, model, opt, metric_keys, train_loader, device, model_name, z_std_norm, y_ids) 
        valid_metrics = evaluate(epoch, model, metric_keys, valid_loader, device, model_name, z_std_norm, y_ids)
        dynamics.append({
            'epoch': epoch,
            'train': train_metrics,
            'valid': valid_metrics
        })

    results = {
        'args': args,
        'dynamics': dynamics,
        'state': model.state_dict(),
        'optimizer': opt.state_dict()
    }

    # save data and metadata to .torch dictionary
    with open(image_dir + '/model.torch', 'wb') as f:
        torch.save(results, f)

### Evaluate a representative model

- Input the **repetition** to load the corresponding trained model.


- Load and process the experimental data.


- Plot the training history.


- Make predictions on all synthetic and experimental data.


- Plot example reconstructions of synthetic data in each error quartile, and reconstructions of experimental data.


- Visualize the latent space along 2 dimensions.


- Plot performance of regressors, if present.


- Find the optimal threshold for classifier and plot classifier performance, if present.

In [ ]:
# repetition to evaluate
r = 0
image_dir = model_dir + '/model_' + str(r)
model, _, _ = init_model(model_name, height, width, num_features, kwargs, device,
                         prox_features=prox_features, scaler=torch_scaler)
model.load_state_dict(torch.load(image_dir + '/model.torch', map_location=device)['state'])
model.eval()

# process experiment
x_exp = process_exp(exp_names, q, x_moms)
x_exp = torch.from_numpy(x_exp).unsqueeze(1)

In [ ]:
# plot training history
dynamics = torch.load(image_dir + '/model.torch', map_location='cpu')['dynamics']
plot_history(image_dir, dynamics, logscale=False)

In [ ]:
# predict on synthetic data
df = get_predictions(model, data_loaders, d_sets, device, height, width, num_features, kwargs, model_name, z_std_norm,
                     y_ids, scaler)

In [ ]:
# predict on experimental data
df_exp = get_predictions_exp(model, x_exp, exp_names, device, height, width, num_features, kwargs, model_name,
                             z_std_norm, y_ids, scaler)

In [ ]:
# convert magnetization units to emu/cm^3
df['y_true_'] = df['y_true'].map(lambda x: convert_magn(x, y_columns))
df['y_pred_'] = df['y_pred'].map(lambda x: convert_magn(x, y_header))
df_exp['y_pred_'] = df_exp['y_pred'].map(lambda x: convert_magn(x, y_header))

# convert density units to g/cm^3
m = [M[0]] + drop_duplicates_list(M[1:])
df['y_true_'] = df['y_true_'].map(lambda x: convert_dens(x, m))
df['y_pred_'] = df['y_pred_'].map(lambda x: convert_dens(x, m))
df_exp['y_pred_'] = df_exp['y_pred_'].map(lambda x: convert_dens(x, m))

In [ ]:
scale = 0
for i in range(len(df_exp)):
    y_pred = df_exp.iloc[i]['y_pred']
    tag = df_exp.iloc[i]['set']
    if i == 0:
        y_lims = plot_SLD(image_dir, sample_name, y_pred, y_header, tag, scale=scale)
    else:
        plot_SLD(image_dir, sample_name, y_pred, y_header, tag, scale=scale, y_lims=y_lims)

In [ ]:
# print predictions of i_th experiment
i = 0
print('experiment:', exp_names[i])
if (model_name == 'rcvae') or (model_name == 'rvae'):
    dict_exp = dict(zip(y_header, df_exp.iloc[i]['y_pred_']))
    print(dict_exp)
    
elif model_name == 'cvae':
    dict_exp = dict(zip(y_header, df_exp.iloc[i]['z'][:len(y_header)-1]))
    dict_exp['class_prox'] = df_exp.iloc[i]['y_pred_'][0]
    print(dict_exp)

else:
    if not z_std_norm:
        dict_exp = dict(zip(y_header, df_exp.iloc[i]['z'][:len(y_header)-1]))
        print(dict_exp)

In [ ]:
# plot example reconstructions
plot_decoded(image_dir, np.stack(df.loc[df['set']=='test', 'x_pred'].values),
             np.stack(df.loc[df['set']=='test', 'x_true'].values),
             np.stack(df.loc[df['set']=='test', 'x_mse'].values), 'test',
             df_exp['x_mse'].values, exp_names)

In [ ]:
# plot reconstructed experiment
plot_decoded_exp(image_dir, np.stack(df_exp['x_pred'].values), exp_names, q, x_moms)

In [ ]:
# visualize separability of latent space
plot_latent_representation(image_dir, np.stack(df.loc[df['set']=='test', 'z'].values),
                           np.stack(df.loc[df['set']=='test', 'y_true_'].values),
                           y_ids, y_labels, y_units_, 'encoded_test', np.stack(df_exp['z'].values), exp_names, mode=2)

In [ ]:
# plot histogram of regressor predictions
if (model_name == 'rcvae') or (model_name == 'rvae'):
    plot_predicted(image_dir, np.stack(df.loc[df['set']=='test', 'y_pred_'].values),
                   np.stack(df.loc[df['set']=='test', 'y_true_'].values), y_ids, y_labels, y_units_)

In [ ]:
# plot regressor performance per property
if (model_name == 'rcvae') or (model_name == 'rvae'):
    for y_name in y_header[:-1]:
        if y_name in ['d_prox', 'magn_prox']: qs = (0.25, 0.5, 0.75, 0.95)
        else: qs = (0.5, 0.75, 0.95)
        plot_predicted_property(image_dir, np.stack(df.loc[df['set']=='test', 'y_pred'].values),
                                np.stack(df.loc[df['set']=='test', 'y_true'].values),
                                np.stack(df.loc[df['set']=='test', 'y_err'].values), y_name,
                                y_header, y_ids, y_labels, y_units, qs=qs)

In [ ]:
# find optimal threshold and plot classifier performance
if 'cvae' in model_name:
    fpr, tpr, roc_auc, th = get_roc(df, d_sets)
    tpr0, fpr0, th = get_optimal_threshold(fpr[1], tpr[1], th[1])

    plot_roc(image_dir, fpr, tpr, roc_auc, [tpr0, fpr0, th])
    plot_precision_recall_f1(image_dir, df, d_sets, th)

In [ ]:
# find optimal threshold for proximity parameters and plot confusion matrices
if (model_name == 'rcvae') or (model_name == 'rvae'):
    for y_name in ['d_prox', 'magn_prox']:
        fpr, tpr, y_th = get_roc_prox(df, y_header, y_name)
        _, _, y_th = get_optimal_threshold(fpr, tpr, y_th)
        plot_confusion_matrix(image_dir, df, y_header, y_name, y_th)

### Evaluate all models

- Compile the training history and experiment predictions on all model repetitions.


- Plot the training history statistics.


- Plot classification statistics on experimental data.

In [ ]:
# predict on all models to compute statistics
dynamics = []
m = [M[0]] + drop_duplicates_list(M[1:])
for r in range(reps):
    image_dir = model_dir + '/model_' + str(r)
    model, _, _ = init_model(model_name, height, width, num_features, kwargs, device,
                             prox_features=prox_features, scaler=torch_scaler)
    model.load_state_dict(torch.load(image_dir + '/model.torch', map_location=device)['state'])
    
    # plot training history
    dynamics += [torch.load(image_dir + '/model.torch', map_location='cpu')['dynamics']]
    
    # get optimal threshold
    if 'cvae' in model_name:
        # predict on synthetic data
        df = get_predictions(model, data_loaders, ['valid'], device, height, width, num_features, kwargs, model_name,
                             z_std_norm, y_ids, scaler)
    
        fpr, tpr, _, th = get_roc(df, ['valid'])
        _, _, th = get_optimal_threshold(fpr[0], tpr[0], th[0])
        
        if model_name == 'rcvae':
            # convert magnetization units to emu/cm^3
            df['y_true_'] = df['y_true'].map(lambda x: convert_magn(x, y_columns))
            df['y_pred_'] = df['y_pred'].map(lambda x: convert_magn(x, y_header))

            # convert density units to g/cm^3
            df['y_true_'] = df['y_true_'].map(lambda x: convert_dens(x, m))
            df['y_pred_'] = df['y_pred_'].map(lambda x: convert_dens(x, m))
        
            # find optimal threshold for proximity parameters
            d_fpr, d_tpr, d_th = get_roc_prox(df, y_header, 'd_prox')
            _, _, d_th = get_optimal_threshold(d_fpr, d_tpr, d_th)

            m_fpr, m_tpr, m_th = get_roc_prox(df, y_header, 'magn_prox')
            _, _, m_th = get_optimal_threshold(m_fpr, m_tpr, m_th)
        
    elif model_name == 'rvae':
        # predict on synthetic data
        df = get_predictions(model, data_loaders, ['valid'], device, height, width, num_features, kwargs, model_name,
                             z_std_norm, y_ids, scaler)
        
        # convert magnetization units to emu/cm^3
        df['y_true_'] = df['y_true'].map(lambda x: convert_magn(x, y_columns))
        df['y_pred_'] = df['y_pred'].map(lambda x: convert_magn(x, y_header))

        # convert density units to g/cm^3
        df['y_true_'] = df['y_true_'].map(lambda x: convert_dens(x, m))
        df['y_pred_'] = df['y_pred_'].map(lambda x: convert_dens(x, m))

        # find optimal threshold for proximity parameters
        d_fpr, d_tpr, d_th = get_roc_prox(df, y_header, 'd_prox')
        _, _, d_th = get_optimal_threshold(d_fpr, d_tpr, d_th)

        m_fpr, m_tpr, m_th = get_roc_prox(df, y_header, 'magn_prox')
        _, _, m_th = get_optimal_threshold(m_fpr, m_tpr, m_th)
    
    # predict on experimental data
    if r > 0:
        df = get_predictions_exp(model, x_exp, exp_names, device, height, width, num_features, kwargs, model_name,
                                 z_std_norm, y_ids, scaler)
        df['model'] = r
        if 'cvae' in model_name:
            df['th'] = th
        if (model_name == 'rcvae') or (model_name == 'rvae'):
            df['d_th'] = d_th
            df['m_th'] = m_th
            df['y_pred_'] = df['y_pred'].map(lambda x: convert_magn(x, y_header))
            df['y_pred_'] = df['y_pred_'].map(lambda x: convert_dens(x, m))
        df_exp = df_exp.append(df, ignore_index=True)
        
    else:
        df_exp = get_predictions_exp(model, x_exp, exp_names, device, height, width, num_features, kwargs, model_name,
                                     z_std_norm, y_ids, scaler)
        df_exp['model'] = r
        if 'cvae' in model_name:
            df_exp['th'] = th
        if (model_name == 'rcvae') or (model_name == 'rvae'):
            df_exp['d_th'] = d_th
            df_exp['m_th'] = m_th
            df_exp['y_pred_'] = df_exp['y_pred'].map(lambda x: convert_magn(x, y_header))
            df_exp['y_pred_'] = df_exp['y_pred_'].map(lambda x: convert_dens(x, m))

In [ ]:
# plot training history statistics
plot_history_statistics(model_dir, dynamics, logscale=False)

In [ ]:
if (model_name == 'rcvae') or (model_name == 'rvae'):
    y_test_orig = scaler.inverse_transform(y_test[:,y_ids[:-1]])
    y_test_ = y_test_orig.copy()
    for i, v in enumerate(y_test_orig):
        y_test_[i,:] = convert_magn(v, y_header)
        y_test_[i,:] = convert_dens(y_test_[i,:], m)
        
    y_min = y_test_.min(axis=0)
    y_max = y_test_.max(axis=0)
    for i, y_name in enumerate(y_header[:-1]):
        if y_name in ['d_prox', 'd_iAFM']:
            y_th = df_exp['d_th'].mean()
        elif 'magn' in y_name:
            y_th = df_exp['m_th'].mean()
        else: y_th = None
        ylims = [y_min[i], y_max[i]]
        plot_exp_statistics(model_dir, df_exp, reps, y_name, y_header, y_labels, y_units_, y_th=y_th, y_lims=ylims)

In [ ]:
# plot class statistics on experimental sample
if 'cvae' in model_name:
    plot_class_exp_statistics(model_dir, df_exp, reps)